### Using LangChain basics

You can use prompt templates to insert data into your prompt easily. Don't forget to create environment variables for your OpenAI or Azure OpenAI keys.

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Using completions API
llm = OpenAI(model_name="text-davinci-003")

# Declare our prompt template.
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# Dynamically pass in our params.
llm_result = llm(prompt.format(product="colorful socks"))

print(llm_result)



Colorful Cozies.


### Using LangChain to get structured data outputs

You can use LangChain to get JSON responses. Below I am wanting my question and answer in a structured format. But you can build whatever models you want.

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

# I am chosing Q&A here, but really you can chose any structure you need for your class. 
# It knows how to generate the class structure based on what you put in description - magic?
class QnA(BaseModel):
    question: str = Field(description="question")
    answer: str = Field(description="answer")
        
actor_query = "What is a good name for a company that makes colorful socks?"

# You can use different parsers, or even construct your own. Pydantic creates nice objects for python so I am using that.
parser = PydanticOutputParser(pydantic_object=QnA)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(query=actor_query)

output = llm(_input.to_string())

parser.parse(output)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


QnA(question='What is a good name for a company that makes colorful socks?', answer='Socktastic!')

### The following demonstrates how to use Conversation History and Memory in a conversation chain

LangChain can help remember what the user is saying for back and forth chatgpt like capabilities

In [3]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)

# The conversation object manages the back and forward conversation, including memory.
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

# We just add user input and let the conversation object handle the rest.
conversation.predict(input="Hi there!")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [ ]:
# Adding more user input to the conversation chain keeps the memory, see the verbose output below.
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

### LangChain Tools

You can set custom tools to be used, for example here is the bing search tool

In [5]:
import os
os.environ["BING_SUBSCRIPTION_KEY"] = ""
os.environ["BING_SEARCH_URL"] = ""

from langchain.utilities import BingSearchAPIWrapper
search = BingSearchAPIWrapper(k=1)
search.run("Dave Enright from Singapore on LinkedIn")

'About. I help companies transform into data driven organisations so they can become predictive instead of reactive. I’ve created a ‘data adoption framework’ that brings organisation from strategy...'

### You can use agents to orchestrate multiple tools together

Below the agent is being told to use the Bing Search tool when it needs to answer current events.The agent will use this tool based on the kinds of questions the user is asking.

In [6]:
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain.utilities import BingSearchAPIWrapper

# The tool description helps langchain work out when to use the tool. Be careful with your descriptions as its not completely deterministic.
search = BingSearchAPIWrapper(k=1)
tools = [
    Tool(
        name = "Intermediate Answer",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True
    )
]

Initialising the agent to use the tool

In [11]:
from typing import List, Tuple, Any, Union
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# There are different kinds of agents documented on the langchain website. This particular one is basically just a chat with search agent.
self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("How many people live in Thailand in 2023?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the population of Thailand in 2020?
Intermediate answer: The current <b>population</b> <b>of Thailand</b> is 70,301,299 as of Wednesday, May 17, 2023, based on ...


> Finished chain.


'The current <b>population</b> <b>of Thailand</b> is 70,301,299 as of Wednesday, May 17, 2023, based on ...'

### Work with Documents

In [7]:
from langchain.document_loaders import PyPDFLoader

# There are different document loaders available, this is the basic PDF loader.
loader = PyPDFLoader("Benefit_Options.pdf")
pages = loader.load_and_split()

In [9]:
# Splitting into pages is useful to give page number back as a source
pages[1]

Document(page_content='This document contains information generated using a language model (Azure OpenAI). The information \ncontained in this document is only for demonstration purposes and does not reflect the opinions or \nbeliefs of Microsoft. Microsoft makes no representations or warranties of any kind, express or implied, \nabout the completeness, accuracy, reliability, suitability or availability with respect to the information \ncontained in this document.  \nAll rights reserved to Microsoft', metadata={'source': 'Benefit_Options.pdf', 'page': 1})

In [9]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# This is creating an index based on embeddings, but as you can see by the output it's a bit ugly to work with.
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("What's the difference between plus and standard?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

3: offers a wider range of prescription drug coverage than Northwind Standard. Both plans offer coverage 
for vision and dental services, as well as medical services.  
Next Steps  
We hope that this information has been helpful in understanding the differe nces between Northwind 
Health Plus and North
2: Welcome to Contoso  Electronics ! We are excited to offer our employees two comprehensive health 
insurance plans through Northwind Health.  
Northwind Health Plus  
Northwind Health Plus is a comprehensive plan that provides comprehensive coverage for medical, 
vision, and dental services. T his pl


In [11]:
from langchain.indexes import VectorstoreIndexCreator

# Using the vector store from the loader means langchain can abstract away a lot of the under the hood stuff.
# Note I am using a local python package for the vectorDB. You may get errors.
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [12]:
# It comes up with better formatted responses.
query = "What's the difference between healthcare plans?"
index.query_with_sources(query)

{'question': "What's the difference between healthcare plans?",
 'answer': ' Northwind Health Plus offers more comprehensive coverage than Northwind Standard, including coverage for emergency services, mental health and substance abuse, and out-of-network services. Northwind Standard does not offer coverage for emergency services, mental health and substance abuse coverage, or out-of-network services.\n',
 'sources': 'Benefit_Options.pdf'}